In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torchvision.utils import save_image
import numpy as np
from sklearn.model_selection import train_test_split
import sys
import random
import os
from os import listdir, mkdir

seed=42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# we use GPU if available, otherwise CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Chargement des images

In [ ]:
# on lit une première fois les images du dataset
# TODO adapter le path selon l'endroit où sont stockées les données
image_directory = "/content/drive/MyDrive/ProjetFromage/data_fromage"
base_dataset = datasets.ImageFolder(image_directory)
test_directory = "/content/drive/MyDrive/ProjetFromage/augmentated_data/test/"
augmentated_directory = "/content/drive/MyDrive/ProjetFromage/augmentated_data/train/"

In [ ]:
si = 0
min = 100
for dir in listdir(image_directory):
  si += len(listdir(image_directory+"/"+dir))
  if len(listdir(image_directory+"/"+dir)) < min:
    min = len(listdir(image_directory+"/"+dir))

print(si)
print(min)

3824
81


In [ ]:
def longueurs():
  satr = 0
  for dir in listdir(augmentated_directory):
    satr += len(listdir(augmentated_directory+dir))

  sate = 0
  for dir in listdir(test_directory):
    sate += len(listdir(test_directory+dir))

  si = 0
  for dir in listdir(image_directory):
    si += len(listdir(image_directory+"/"+dir))

  print("Longueur dataset de base : "+str(si))
  print("Longueur dataset de test : "+str(sate)+" soit "+str(round(100*sate/(sate+satr)))+"%")
  print("Longueur dataset augmenté : "+str(satr)+" soit "+str(round(satr/(si-sate)))+" fois plus")

In [ ]:
# Normalisation des images pour les modèles pré-entraînés PyTorch
# voir: https://pytorch.org/docs/stable/torchvision/models.html
# et ici pour les « explications » sur les valeurs exactes: https://github.com/pytorch/vision/issues/1439
# On va essayer sans normaliser car cela modifier beaucoup les couleurs
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_normalize = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor()
])

np.random.seed(42)

dataset_full = datasets.ImageFolder(image_directory, data_normalize)
samples_train, samples_test = train_test_split(dataset_full.samples)

dataset_train = datasets.ImageFolder(image_directory, data_normalize)
dataset_train.samples = samples_train
dataset_train.imgs = samples_train
loader_train = torch.utils.data.DataLoader(dataset_train)

dataset_test = datasets.ImageFolder(image_directory, data_normalize)
dataset_test.samples = samples_test
dataset_test.imgs = samples_test
loader_test = torch.utils.data.DataLoader(dataset_test)

torch.manual_seed(42)

In [ ]:
label_to_name = [f for f in listdir(image_directory)]

In [ ]:
for label in label_to_name:
  mkdir(augmentated_directory+label)
  mkdir(test_directory+label)

In [ ]:
for i, img in enumerate(loader_test):
  save_image(img[0], test_directory+label_to_name[img[1]]+'/'+str(i)+'.png')

In [ ]:
for i, img in enumerate(loader_train):
  save_image(img[0], augmentated_directory+label_to_name[img[1]]+'/'+str(i)+'.png')

In [ ]:
longueurs()

Longueur dataset de base : 3824
Longueur dataset de test : 954 soit 25%
Longueur dataset augmenté : 2860 soit 1 fois plus


# Affichage des images

In [ ]:
iterator = range (0, 2000, 200)

Pour les tenseurs :

In [ ]:
def tensor_imshow(tensor, title=None):
    img = tensor.cpu().clone()
    img = img.squeeze()
    img = img.permute(1 , 2 , 0)
    plt.imshow(img, interpolation='nearest')
    if title is not None:
        plt.title(title)  
    plt.pause(0.5)

Affichage de nb images-tenseurs aléatoires :

In [ ]:
def random_display_tensor(dataset, nb=10):

    plt.figure()

    for ii in random.sample(range(len(dataset)), nb):
        tensor_imshow(dataset[ii][0] , title=label_to_name[dataset[ii][1]])

    plt.close()

In [ ]:
def display_tensor(dataset, nb=10):

    plt.figure()

    for ii in range (nb):
        tensor_imshow(dataset[ii][0] , title=label_to_name[dataset[ii][1]])

    plt.close()

Pour les images PIL, affichage d'images aléatoire :

In [ ]:
def random_display_pil(dataset, nb=10):

    plt.figure()

    for ii in random.sample(range(len(dataset)), nb):
        plt.imshow(dataset[ii][0])
        plt.title(label_to_name[dataset[ii][1]])
        plt.pause(0.5)

    plt.close()

# Transformations

## Rotate

In [ ]:
def rotate_transform(dataset, iterator=None):
    if iterator is None:
        iterator = range(0, len(dataset))
    rotater = transforms.RandomRotation(degrees=(0, 360))
    rotated_imgs = []
    for i in iterator :
        img = dataset[i]
        tr_img = rotater(img[0])
        rotated_imgs.append((tr_img, img[1]))
        save_image(tr_img, augmentated_directory+label_to_name[img[1]]+'/rotated_'+str(i)+'.png')
    return rotated_imgs

In [ ]:
#random_display_tensor(rotate_transform(normalized_dataset, random.sample(range(len(normalized_dataset)), 10)))
#display_tensor(rotate_transform(normalized_dataset, iterator))

## Flip

In [ ]:
def flip_transform(dataset, iterator=None):
    if iterator is None:
        iterator = range(0, len(dataset))
    v_flipper = transforms.RandomVerticalFlip(p=1)
    h_flipper = transforms.RandomHorizontalFlip(p=1)
    flipped_imgs = []
    j = 0
    for i in iterator :
        flipper = [v_flipper, h_flipper][j%2]
        img = dataset[i]
        tr_img = flipper(img[0])
        flipped_imgs.append((tr_img, img[1]))
        save_image(tr_img, augmentated_directory+label_to_name[img[1]]+'/flipped_'+str(i)+'.png')
        j += 1
    return flipped_imgs

In [ ]:
#random_display_tensor(flip_transform(normalized_dataset, random.sample(range(len(normalized_dataset)), 10)))
#display_tensor(flip_transform(normalized_dataset, iterator))

## Resize

In [ ]:
def resize_transform(dataset, iterator=None):
    if iterator is None:
        iterator = range(0, len(dataset))
    resizer = transforms.Resize(random.randint(100, 100))
    resized_imgs = []
    for i in iterator :
        img = dataset[i]
        tr_img = transforms.Resize(224)(resizer(img[0]))
        resized_imgs.append((tr_img , img[1]))
        save_image(tr_img, augmentated_directory+label_to_name[img[1]]+'/resized_'+str(i)+'.png')
    return resized_imgs

In [ ]:
#random_display_tensor(resize_transform(normalized_dataset, random.sample(range(len(normalized_dataset)), 10)))
#display_tensor(resize_transform(normalized_dataset, iterator))

## Color

In [ ]:
def color_transform(dataset, iterator=None):
    if iterator is None:
        iterator = range(0, len(dataset))
    low_colorer = transforms.ColorJitter(brightness=(.7, .7), contrast=(.7, .7), saturation=(.2, .2), hue=0)
    high_colorer = transforms.ColorJitter(brightness=(1.35, 1.35), contrast=(1.3, 1.3), saturation=(2, 2), hue=0)
    hue_colorer = transforms.ColorJitter(brightness=0, contrast=0, saturation=0, hue=.06)
    colored_imgs = []
    j = 0
    for i in iterator :
        colorer = [low_colorer, high_colorer, hue_colorer][j%3]
        img = dataset[i]
        tr_img = colorer(img[0])
        colored_imgs.append((tr_img, img[1]))
        save_image(tr_img, augmentated_directory+label_to_name[img[1]]+'/colored_'+str(i)+'.png')
        j += 1
    return colored_imgs

In [ ]:
#random_display_tensor(color_transform(dataset_train, random.sample(range(len(dataset_train)), 10)))
#display_tensor(color_transform(normalized_dataset, iterator))

## Crop

In [ ]:
def crop_transform(dataset, iterator=None):
    if iterator is None:
        iterator = range(0, len(dataset))
    cropper = transforms.RandomCrop(size=(158, 158))
    cropped_imgs = []
    for i in iterator :
        img = dataset[i]
        tr_img = transforms.Resize(224)(cropper(img[0]))
        cropped_imgs.append((tr_img, img[1]))
        save_image(tr_img, augmentated_directory+label_to_name[img[1]]+'/cropped_'+str(i)+'.png')
    return cropped_imgs

In [ ]:
#random_display_tensor(crop_transform(normalized_dataset, random.sample(range(len(normalized_dataset)), 10)))
#display_tensor(crop_transform(normalized_dataset, iterator))

# Création d'un jeu de données augmenté

In [ ]:
rotate_transform(dataset_train)
print("Rotate done")
flip_transform(dataset_train)
print("Flip done")
resize_transform(dataset_train)
print("Resize done")
color_transform(dataset_train)
print("Color done")
crop_transform(dataset_train)
print("Crop done")

Rotate done
Flip done
Resize done
Color done
Crop done


In [ ]:
longueurs()

Longueur dataset de base : 3824
Longueur dataset de test : 954 soit 5%
Longueur dataset augmenté : 17160 soit 6 fois plus
